In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss

from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train[0:12].T

In [ ]:
train2=train.drop('id',axis=1)
test2=test.drop('id',axis=1)

In [ ]:
Name0=train2['target'].unique()
Name=sorted(Name0)
print(Name)

In [ ]:
N=[]
for i in range(9):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

print(normal_mapping)

In [ ]:
X = train2.drop('target',axis=1)
y = train2['target'].map(normal_mapping)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
def cross_val(X, y, model, params, folds=5):

    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=21)
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f"Fold: {fold}")
        x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        x_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

        alg = model(**params)
        alg.fit(x_train, y_train,
                eval_set=[(x_test, y_test)],
                early_stopping_rounds=100,
                verbose=400)

        pred = alg.predict_proba(x_test)
        loss = log_loss(y_test, pred)
        print(f"Log loss: {loss}")
        print("-"*50)
    
    return alg

In [ ]:
xgb_params= {'n_estimators': 20000, 
             'max_depth': 8, 
             'learning_rate': 0.04, 
             'reg_lambda': 28, 
             'subsample': 0.8, 
             'colsample_bytree': 0.3, 
             'colsample_bynode': 0.8, 
             'colsample_bylevel': 0.4}

In [ ]:
xgb_model = cross_val(X,y,XGBClassifier,xgb_params)

In [ ]:
pred_xgb = xgb_model.predict_proba(test2)

In [ ]:
print(pred_xgb.shape)
print(pred_xgb[0])

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")
sample

In [ ]:
subm = sample
subm[ Name ] = pred_xgb
subm.to_csv('submission.csv',index=False)
subm